In [3]:
from mohan.Similarity import Similarity
from bson.objectid import ObjectId
from pymongo import MongoClient
import random

import json

from re import sub
from unidecode import unidecode
from bs4 import BeautifulSoup
from thefuzz import process, fuzz

import pandas as pd
from elasticsearch import Elasticsearch

In [ ]:
# Check for ElasticSearch indexes
es = Elasticsearch("http://localhost:9200", basic_auth=('elastic', 'XXXXX'), request_timeout=120)
es.indices.get_alias().keys()

In [12]:
#s = Similarity("openalex_index",es_uri= "http://localhost:9200", es_auth = ('elastic', 'XXXXX'))
#s.delete_index(index_name="openalex_index")

In [5]:
# MongoDB connections
db_uri="mongodb://localhost:27017/"

dbclient = MongoClient(db_uri)
oa_db = dbclient["openalexco_new"]
openalex_works = oa_db['works']

In [ ]:
# Insert Opennalex works to ElasticSearch
es_index = "openalex_index"

#c reating the instance
s = Similarity(es_index,es_uri= "http://localhost:9200", es_auth = ('elastic', 'XXXXX'))

# taking openalex as example.
openalex = list(openalex_works.find({}, {"_id": 1, "title": 1, "display_name": 1, "primary_location": 1, "publication_year": 1, "biblio": 1, "authorships": 1}))
total_documents = openalex_works.count_documents({})
#example inserting documents to the Elastic Search index.
bulk_size = 100
es_entries = []
counter = 0
for i in openalex:
    work = {}
    work["title"] = i["title"]
    if "primary_location" in i.keys() and i["primary_location"]:
        if i["primary_location"]["source"]:
            work["source"] = i["primary_location"]["source"]["display_name"]
        work["source"] = ""
    else:
        work["source"] = ""
    work["year"] = i["publication_year"]
    work["volume"] = i["biblio"]["volume"]
    work["issue"] = i["biblio"]["issue"]
    work["first_page"] = i["biblio"]["first_page"]
    work["last_page"] = i["biblio"]["last_page"]
    authors = []
    for author in i['authorships']:
        if "display_name" in author["author"].keys():
            authors.append(author["author"]["display_name"])
    work["authors"] = authors
    
    entry = {"_index": es_index,
                "_id": str(i["_id"]),
                "_source": work}
    es_entries.append(entry)
    if len(es_entries) == bulk_size:
        s.insert_bulk(es_entries)
        es_entries = []

print(f"Inserted {len(openalex)} documents from {total_documents}")

In [6]:
# Load minciencias sample
with open("minciencias_noauthors.json", "r") as r:
    j = json.load(r)

In [7]:
j[0]

{'titles': [{'title': 'La enseñanza en economía: una reflexión desde la perspectiva del contenido',
   'lang': 'es',
   'source': 'minciencias'}],
 'updated': [{'source': 'minciencias', 'time': 1711049323}],
 'subtitle': '',
 'abstract': '',
 'keywords': [],
 'types': [{'provenance': 'minciencias',
   'source': 'minciencias',
   'type': 'Artículos de investigación'}],
 'external_ids': [{'provenance': 'minciencias',
   'source': 'minciencias',
   'id': 'ART-0000000007-13'},
  {'provenance': 'minciencias', 'source': 'COD_RH', 'id': '0000000007'},
  {'provenance': 'minciencias', 'source': 'COD_PRODUCTO', 'id': '13'}],
 'external_urls': [],
 'date_published': None,
 'year_published': None,
 'bibliographic_info': {},
 'references_count': None,
 'references': [],
 'citations_count': [],
 'citations': [],
 'author_count': 1,
 'authors': [{'full_name': '',
   'affiliations': [{'id': '65f93c212685775e89fdd8ce',
     'name': 'Grupo de Desarrollo y Equidad',
     'types': [{'source': 'minciencias

In [8]:
def flatten(xss):
    if all(xs is None for xs in xss):
        return []
    return [x for xs in xss for x in xs]

In [15]:
found = []

for i in range(len(j)):
    results = []
    titles = []
    original_title = j[i].get('titles')[0].get('title')
    # Remove special OpenAlex API special characters
    title = sub('[_\|,]','',original_title).lower()
    # print(title)
    
    es_results = s.search_work(title=title, source = "", year = 0, authors = [], volume = "", issue = "", page_start = "", page_end = "", use_es_thold = True, es_thold=0, hits=20)
    if not es_results:
        # print("Not result")
        continue
    for es_work in es_results:
        work_candidate = openalex_works.find_one({"_id": ObjectId(es_work["_id"])})
        if work_candidate:
            results.append(work_candidate)
    
    # remove words with unicode character is there is not match
    titles = [result.get('display_name') for result in results]
    #Check mathml in titles and UPDATE display_name value!
    for ii in range(len(titles)):
        if [tag.name for tag in BeautifulSoup(titles[ii]).find_all() if tag.name.find('math')>-1]:
            titles[ii] =sub(r'\s+',' ', sub('\n',' ', BeautifulSoup(sub(r"([a-zA-Z])<", r"\1 <",titles[ii])).text.strip()))
            results[ii]['display_name'] = titles[ii]
    # Get better match
    display_name, score = process.extractOne(original_title,titles)
    # store if score sufficiently high
    result = [d for d in results if d.get('display_name')==display_name and score > 94]
    
    #If not author name avalaible → check for country 'CO'
    if result:
        result = result[0]
        countries = [d.get('countries') for d in result.get('authorships')]
        # print(countries)
        if countries:
            countries = flatten(countries)
        if 'CO' in list(set(countries)):
            result['minciencias'] = j[i]
            result['original_title'] = original_title
            result['score_thefuzz'] = score
            #append result
            print(f"minciencias_title: {original_title}\n openalexdb_title: {display_name}\n")
            
            found.append(result)

/tmp/ipykernel_3522351/45239626.py:24: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 24 of the file /home/fvergara/notebooks/Untitled.ipynb. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  if [tag.name for tag in BeautifulSoup(titles[ii]).find_all() if tag.name.find('math')>-1]:


minciencias_title: La enseñanza en economía: una reflexión desde la perspectiva del contenido
 openalexdb_title: La enseñanza en economía: una reflexión desde la perspectiva del contenido

minciencias_title: Incidencia del crecimiento económico en las emisiones de CO2Ñ Un análisis comparativo (1980-2008)
 openalexdb_title: Incidencia del crecimiento económico en las emisiones de CO² : un análisis comparativo (1980-2008)

minciencias_title: La enseñanza de la Economía: reflexiones acerca del sentido de una ciencia social en construcción. El caso colombiano
 openalexdb_title: La enseñanza de la economía: reflexiones acerca del sentido de una ciencia social en construcción. el caso colombiano

minciencias_title: La enseñanza de la economía: Reflexiones acerca del sentido de una ciencia social en construcción.
 openalexdb_title: La enseñanza de la economía: reflexiones acerca del sentido de una ciencia social en construcción. el caso colombiano

minciencias_title: Susceptibilidad de Aede

In [14]:
len(found)

50

In [16]:
pd.DataFrame(found)[['display_name','original_title','score_thefuzz']]

,display_name,original_title,score_thefuzz
0,La enseñanza en economía: una reflexión desde ...,La enseñanza en economía: una reflexión desde ...,100
1,Incidencia del crecimiento económico en las e...,Incidencia del crecimiento económico en las em...,96
2,La enseñanza de la economía: reflexiones acerc...,La enseñanza de la Economía: reflexiones acerc...,100
3,La enseñanza de la economía: reflexiones acerc...,La enseñanza de la economía: Reflexiones acerc...,95
4,Susceptibilidad de Aedes aegypti (Diptera: Cul...,Susceptibilidad de Aedes aegypti (Diptera: Cul...,100
5,Instrumentos de política para la gestión de se...,Instrumentos de política para la gestión de se...,100
6,La regulación de telefonía local en Colombia: ...,La Regulación de Telefonía Local en Colombia: ...,100
7,La economía experimental como aprendizaje sign...,La economía experimental como aprendizaje sign...,100
8,Competitividad: Una condición y orbásica para ...,Competitividad: una condición básica para el c...,96
9,Aportes Del “Nonágono semiótico” Para la compr...,"Aportes del ""nonágono semiótico"" para la compr...",99


In [ ]:
pd.DataFrame(found).to_csv('found_es.csv', index=False) 